In [1]:
!pip install transformers==4.30
!pip install accelerate -U
!pip install sentencepiece
!pip install rouge
!pip install wandb onnx -Uq

In [2]:
!git clone https://github.com/Arjavjain100/TOS-Summarization.git

fatal: destination path 'TOS-Summarization' already exists and is not an empty directory.


In [3]:
from transformers import LEDForConditionalGeneration, LEDTokenizer, Trainer, TrainingArguments,pipeline,PretrainedConfig
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.model_selection import train_test_split
from rouge import Rouge
import pandas as pd
import os
import wandb
import random
import numpy as np
import accelerate

os.environ["WANDB_PROJECT"]="major-one"
os.environ["WANDB_LOG_MODEL"]="checkpoint"
os.environ["WANDB_WATCH"]="all"



# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Dataset location
filename = "./TOS-Summarization/Dataset/all_v1_transpose.csv"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
!pip install wandb -U

In [7]:
!wandb login()

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `wandb login()'


In [8]:
df = pd.read_csv(filename)
df = df[['original_text','reference_summary']]
df.rename(columns = {'original_text':'source', 'reference_summary':'target'}, inplace = True)
len(df)

446

In [9]:
X = df['source']
y = df['target']

In [10]:
df.head()

,source,target
0,welcome to the pokémon go video game services ...,hi.
1,by using our services you are agreeing to thes...,by playing this game you agree to these terms....
2,if you want to use certain features of the ser...,you have to use google pokemon trainer club or...
3,during game play please be aware of your surro...,don t die or hurt others and if you do it s no...
4,subject to your compliance with these terms ni...,don t copy modify resell distribute or reverse...


In [11]:
class LEDDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])  # torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels['input_ids'])

In [12]:
def prepare_data(model_name,
                 train_texts, train_labels,
                 test_texts, test_labels):
  """
  Prepare input data for model fine-tuning
  """

  tokenizer = AutoTokenizer.from_pretrained(model_name)
  prepare_test = False if test_texts is None or test_labels is None else True

  def tokenize_data(texts, labels):

    encodings = tokenizer(texts, truncation=True, padding=True, max_length = 600)
    decodings = tokenizer(labels, truncation=True, padding=True, max_length = 256)
    dataset_tokenized = LEDDataset(encodings, decodings)
    return dataset_tokenized

  train_dataset = tokenize_data(train_texts, train_labels)
  test_dataset = tokenize_data(test_texts, test_labels) if prepare_test else None

  return train_dataset, test_dataset, tokenizer

In [16]:
def prepare_fine_tuning(model_name, tokenizer, train_dataset, test_dataset, freeze_encoder=False, output_dir='./results'):
  """
  Prepare configurations and base model for fine-tuning
  """
  torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = LEDForConditionalGeneration.from_pretrained(model_name).to(torch_device)

  if test_dataset is not None:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=2,              # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      per_device_eval_batch_size=1,    # batch size for evaluation, can increase if memory allows
      save_steps=500,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      evaluation_strategy='steps',     # evaluation strategy to adopt during training
      eval_steps=100,                  # number of update steps before evaluation
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=100,
      report_to="wandb",
      run_name = "longformer"
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=test_dataset,           # evaluation dataset
      tokenizer=tokenizer
    )

  else:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=2,              # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      save_steps=500,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=100,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      tokenizer=tokenizer
    )

  return trainer

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

train_texts, train_labels = list(X_train), list(y_train)
test_texts, test_labels = list(X_test), list(y_test)

In [18]:
model_name = 'allenai/led-base-16384'

train_dataset,test_dataset, tokenizer = prepare_data(model_name, train_texts, train_labels,test_texts,test_labels)
trainer = prepare_fine_tuning(model_name, tokenizer, train_dataset,test_dataset)

trainer.train()

trainer.evaluate(test_dataset)

wandb.finish()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: aarushi-jain211 (faltu-team). Use `wandb login --relogin` to force relogin


You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
100,9.902600,5.044665
200,3.150700,1.118686
300,0.845200,0.460910


eval/loss,█▂▁▁
eval/runtime,▁██▂
eval/samples_per_second,█▁▁▇
eval/steps_per_second,█▁▁▇
train/epoch,▁▁▄▄▆▆██
train/global_step,▁▁▄▄▆▆██
train/learning_rate,▁▅█
train/loss,█▃▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [19]:
import os
if not os.path.exists('./ouput_model/'):
    os.makedirs('./ouput_model/')
trainer.model.save_pretrained("./ouput_model/")

Inference

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [21]:
config = PretrainedConfig.from_json_file('./ouput_model/config.json')

In [23]:
model = LEDForConditionalGeneration.from_pretrained("./ouput_model/").to(device)

In [24]:
def summarize(text):
  input_tokenized = tokenizer.encode(text, return_tensors='pt',max_length=1024,truncation=True).to(device)
  summary_ids = model.generate(input_tokenized,
                                  num_beams=9,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2.0,
                                  min_length=50,
                                  max_length=150,
                                  early_stopping=True)
  summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]

  return summary

In [25]:
y_pred = X_test.apply(lambda x: summarize(x))

In [26]:
summary = pd.concat([y_test.to_frame(name="reference_summary"), y_pred.to_frame(name="generated_summary")], axis=1)

In [27]:
rouge = Rouge()

In [28]:
rouge.get_scores(summary['generated_summary'], summary['reference_summary'],avg=True)

{'rouge-1': {'r': 0.5231165604311384,
  'p': 0.207027982213309,
  'f': 0.28370216792727043},
 'rouge-2': {'r': 0.2462429622268355,
  'p': 0.07235508415967715,
  'f': 0.10572520256578813},
 'rouge-l': {'r': 0.46197352354790544,
  'p': 0.1807507376035193,
  'f': 0.24799084745873354}}